In [1]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [ ]:
%pip install ultralytics

In [12]:
from ultralytics import YOLO

model = YOLO("yolo11x.pt")

results = model(['./data/training/0001/cropped_panos/back_floor_01_partial_room_08_pano_22.jpg'])

for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen


0: 640x640 (no detections), 87.5ms
Speed: 2.2ms preprocess, 87.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)


In [12]:
import os
import json
import numpy as np
from glob import glob
import shutil
from tqdm import tqdm
import ntpath

def convert_polygon_to_bbox(points):
    """Konvertiert Polygon-Punkte in eine Bounding Box [xmin, ymin, xmax, ymax]"""
    points = np.array(points)
    xmin, ymin = np.min(points, axis=0)
    xmax, ymax = np.max(points, axis=0)
    return [xmin, ymin, xmax, ymax]

def convert_labelme_to_yolo(json_dir, image_dir, output_dir, class_mapping=None):
    """
    Konvertiert LabelMe-Annotationen zu YOLO-Format.
    
    Args:
        json_dir: Verzeichnis mit den JSON-Annotationsdateien
        image_dir: Verzeichnis mit den Originalbildern (übergeordnetes Verzeichnis)
        output_dir: Ausgabeverzeichnis für das YOLO-Dataset
        class_mapping: Wörterbuch zum Mapping von Klassennamen zu Indizes (optional)
    """
    # Erstelle Ausgabeverzeichnisse
    labels_dir = os.path.join(output_dir, 'labels')
    images_dir = os.path.join(output_dir, 'images')
    os.makedirs(labels_dir, exist_ok=True)
    os.makedirs(images_dir, exist_ok=True)
    
    # Finde alle JSON-Dateien
    json_files = glob(os.path.join(json_dir, '**', '*.json'), recursive=True)
    print(f"Gefunden: {len(json_files)} Annotationsdateien")
    
    # Wenn kein Klassen-Mapping angegeben ist, erstelle es dynamisch
    if class_mapping is None:
        class_mapping = {}
        class_index = 0
        
        print("Sammle Klassennamen aus Annotationen...")
        for json_file in tqdm(json_files):
            try:
                with open(json_file, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    
                for shape in data.get('shapes', []):
                    label = shape.get('label')
                    if label and label not in class_mapping:
                        class_mapping[label] = class_index
                        class_index += 1
            except Exception as e:
                print(f"Fehler beim Lesen von {json_file}: {e}")
                
        print(f"Gefundene Klassen: {class_mapping}")
    
    # Konvertiere jede JSON-Datei
    processed_files = 0
    failed_files = 0
    print("Konvertiere Annotationen...")
    for json_file in tqdm(json_files):
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # Bildpfad und -größe
            image_filename = data.get('imagePath')
            if not image_filename:
                print(f"Keine imagePath in {json_file}")
                continue
            
            # Extrahiere nur den Dateinamen aus dem Pfad
            image_basename = ntpath.basename(image_filename)
            
            # Versuche das Bild auf mehrere Arten zu finden:
            
            # 1. Direkter Pfad aus JSON
            src_image_path = os.path.normpath(os.path.join(os.path.dirname(json_file), image_filename))
            
            # 2. Im image_dir
            if not os.path.exists(src_image_path):
                src_image_path = os.path.join(image_dir, image_basename)
            
            # 3. Rekursive Suche im image_dir
            if not os.path.exists(src_image_path):
                possible_paths = glob(os.path.join(image_dir, '**', image_basename), recursive=True)
                if possible_paths:
                    src_image_path = possible_paths[0]
            
            # 4. Versuche, relative Pfade zu lösen (../cropped_panos/...)
            if not os.path.exists(src_image_path) and '..' in image_filename:
                # Löse relativen Pfad relativ zum JSON-Verzeichnis
                json_dir_path = os.path.dirname(json_file)
                relative_path = os.path.normpath(os.path.join(json_dir_path, image_filename))
                if os.path.exists(relative_path):
                    src_image_path = relative_path
            
            # Wenn Bild immer noch nicht gefunden, überspringe diese Annotation
            if not os.path.exists(src_image_path):
                print(f"Bild nicht gefunden für {json_file}. Pfad: {image_filename}")
                failed_files += 1
                continue
            
            # Kopiere das Bild in das Ausgabeverzeichnis
            dst_image_path = os.path.join(images_dir, image_basename)
            shutil.copy(src_image_path, dst_image_path)
            
            # Hole Bildgröße entweder aus JSON oder aus dem Bild
            if 'imageWidth' in data and 'imageHeight' in data:
                img_width = data['imageWidth']
                img_height = data['imageHeight']
            else:
                # Wenn Größe nicht in JSON, verwende PIL
                from PIL import Image
                try:
                    with Image.open(dst_image_path) as img:
                        img_width, img_height = img.size
                except Exception as e:
                    print(f"Fehler beim Lesen der Bildgröße für {image_basename}: {e}")
                    continue
            
            # Erstelle YOLO-Annotationsdatei
            base_filename = os.path.splitext(image_basename)[0]
            txt_filename = base_filename + '.txt'
            txt_path = os.path.join(labels_dir, txt_filename)
            
            with open(txt_path, 'w') as f:
                for shape in data.get('shapes', []):
                    label = shape.get('label')
                    shape_type = shape.get('shape_type', 'polygon')
                    points = shape.get('points', [])
                    
                    if not label or not points:
                        continue
                    
                    if label not in class_mapping:
                        print(f"Warnung: Klasse '{label}' nicht im Mapping gefunden. Wird übersprungen.")
                        continue
                    
                    class_id = class_mapping[label]
                    
                    # Konvertiere Punkte zu Bounding Box
                    if shape_type == 'polygon' or shape_type == 'rectangle':
                        xmin, ymin, xmax, ymax = convert_polygon_to_bbox(points)
                    else:
                        # Andere Shape-Typen werden nicht unterstützt
                        continue
                    
                    # Berechne zentrierten x, y, Breite und Höhe (normalisiert)
                    center_x = (xmin + xmax) / 2 / img_width
                    center_y = (ymin + ymax) / 2 / img_height
                    width = (xmax - xmin) / img_width
                    height = (ymax - ymin) / img_height
                    
                    # Begrenze Werte auf den Bereich [0, 1]
                    center_x = max(0, min(1, center_x))
                    center_y = max(0, min(1, center_y))
                    width = max(0, min(1, width))
                    height = max(0, min(1, height))
                    
                    # Schreibe ins YOLO-Format
                    f.write(f"{class_id} {center_x:.6f} {center_y:.6f} {width:.6f} {height:.6f}\n")
            
            processed_files += 1
            
        except Exception as e:
            print(f"Fehler bei Datei {json_file}: {e}")
            failed_files += 1
    
    # Erstelle YAML-Datei
    class_names = [""] * len(class_mapping)
    for name, idx in class_mapping.items():
        class_names[idx] = name
    
    yaml_content = f"""
# LabelMe zu YOLO konvertiert
train: {os.path.join(output_dir, 'images')}
val: {os.path.join(output_dir, 'images')}  # Anpassen für getrennte Validierungsdaten

# Klassenanzahl
nc: {len(class_names)}

# Klassennamen
names: {class_names}
"""
    
    with open(os.path.join(output_dir, 'dataset.yaml'), 'w') as f:
        f.write(yaml_content)
    
    print(f"Konvertierung abgeschlossen! {processed_files} Dateien erfolgreich verarbeitet.")
    print(f"Fehlgeschlagene Dateien: {failed_files}")
    print(f"YAML-Datei erstellt: {os.path.join(output_dir, 'dataset.yaml')}")
    
    return os.path.join(output_dir, 'dataset.yaml'), class_mapping

# Beispielverwendung
if __name__ == "__main__":
    # Setze diese Pfade entsprechend deinem Setup
    json_dir = "data/training/0000/cropped_masks/"      # Verzeichnis mit den JSON-Dateien
    image_dir = "data/training/0000/cropped_panos/"                      # Übergeordnetes Verzeichnis mit allen Bildern
    output_dir = "yolo2"                       # Ausgabeverzeichnis
    
    # Optional: Definiere ein festes Klassen-Mapping
    class_mapping = {
        "window": 0,
        "door": 1
    }
    
    yaml_path, class_map = convert_labelme_to_yolo(json_dir, image_dir, output_dir, class_mapping)

Gefunden: 128 Annotationsdateien
Konvertiere Annotationen...


100%|██████████| 128/128 [00:01<00:00, 124.29it/s]

Konvertierung abgeschlossen! 128 Dateien erfolgreich verarbeitet.
Fehlgeschlagene Dateien: 0
YAML-Datei erstellt: yolo2\dataset.yaml


    json_dir = "data/training/0001/cropped_masks/"      # Verzeichnis mit den JSON-Dateien
    image_dir = "data/training/0001/cropped_panos/" 

In [16]:
from ultralytics import YOLO

# Lade ein vortrainiertes Modell
model = YOLO('./runs/detect/train4/weights/best.pt')  # oder yolov8s.pt, yolov8m.pt, etc.

# Starte Training
results = model.train(
    data='yolo2/dataset.yaml',  # Pfad zur YAML-Datei
    epochs=100,                   # Anzahl der Durchläufe
    imgsz=640,                    # Bildgröße für Training
    batch=16,                     # Batch-Größe (an deine GPU anpassen)
    patience=50,                  # Early Stopping
    device=0,                     # GPU-ID (0 für erste GPU)
)

# Validiere das Modell
metrics = model.val()

# Speichere das Modell
model.export(format='onnx')  # Exportiere in ONNX-Format

Ultralytics 8.3.87  Python-3.12.8 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
engine\trainer: task=detect, mode=train, model=./runs/detect/train4/weights/best.pt, data=yolo2/dataset.yaml, epochs=100, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tru

train: Scanning G:\# Semester 3\Objekterkennung\floorscan\yolo2\labels.cache... 128 images, 53 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
val: Scanning G:\# Semester 3\Objekterkennung\floorscan\yolo2\labels.cache... 128 images, 53 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]


Plotting labels to runs\detect\train5\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train5
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100       2.3G        1.1      2.052      1.267         34        640: 100%|██████████| 8/8 [00:01<00:00,  5.19it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  8.51it/s]

                   all        128        121       0.69      0.707      0.742      0.564


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  6.92it/s]


                   all        128        121      0.995      0.984      0.995      0.958
                window         46         59      0.994          1      0.995      0.962
                  door         46         62      0.997      0.968      0.994      0.954
Speed: 0.3ms preprocess, 1.0ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect\train5
Ultralytics 8.3.87  Python-3.12.8 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning G:\# Semester 3\Objekterkennung\floorscan\yolo2\labels.cache... 128 images, 53 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:06<00:00,  1.15it/s]


                   all        128        121      0.995      0.984      0.995      0.956
                window         46         59      0.993          1      0.995      0.962
                  door         46         62      0.997      0.968      0.994      0.951
Speed: 0.7ms preprocess, 4.0ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect\train52
Ultralytics 8.3.87  Python-3.12.8 torch-2.3.1+cu121 CPU (AMD Ryzen 5 7600X 6-Core Processor)

PyTorch: starting from 'runs\detect\train5\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (6.0 MB)
ONNX: export failure  7.2s: DLL load failed while importing onnx_cpp2py_export: Eine DLL-Initialisierungsroutine ist fehlgeschlagen.


ImportError: DLL load failed while importing onnx_cpp2py_export: Eine DLL-Initialisierungsroutine ist fehlgeschlagen.

In [6]:
from ultralytics import YOLO 

model = YOLO("./runs/detect/train4/weights/best.pt")

In [11]:
img = "./data/condensed/testing/0002/back_floor_01_partial_room_10_pano_49.jpg"


results = model([img])  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # o screen


0: 640x640 2 windows, 3.8ms
Speed: 2.1ms preprocess, 3.8ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)


In [1]:
model.export(format='onnx')

NameError: name 'model' is not defined